In [ ]:
!pip install Wooldridge
!pip install linearmodels

In [ ]:
import pandas as pd
import numpy as np
import wooldridge as wd
import statsmodels.api as sm
from linearmodels.iv import IV2SLS

def Teste_F_Rapido_Robusto(H0, Nivel_de_Significância = 0.05):
    '''
    Função que calcula um teste F de forma mais rápida com base nas restrições de H0, podendo ser robusto se o Resultado for fruto de uma regressão robusta.
    H0 deve estar na forma B1 = B2 =...= Valor que deseja ser testado (0 na maioria das vezes)
    '''
    global Resultado
    ## A função utiliza o método wald_test dos resultados das regressões
    # Para modelos de painel - cujo método usa a estatística LM -, devemos especificar o parâmetro 'formula', o que não ocorre com cortes transversais
    try:
        teste = 'LM'
        est = Resultado.wald_test(formula=H0).stat
        p = Resultado.wald_test(formula=H0).pval
    except:
        teste = 'F'
        est = float(str(Resultado.wald_test(H0))[19:29])
        p = float(str(Resultado.wald_test(H0))[36:47])

    if Nivel_de_Significância > p:
        print(f"O valor de {teste} é {round(est,6)} e seu p-valor é {round(p,7)}.\nPortanto, rejeita-se Ho à significância de {Nivel_de_Significância*100}%, ou seja, as variáveis são conjuntamente significantes.")
    else:
        print(f"O valor de {teste} é {round(est,6)} e seu p-valor é {round(p,7)}.\nPortanto, NÃO se rejeita Ho à significância de {Nivel_de_Significância*100}%, ou seja, as variáveis NÃO são conjuntamente significantes.")

def Regressao_IV_MQ2E(exog, endog, instrumentos, y, constante="S",cov='normal'):
    global df, Resultado
    ## formando o vetor de variáveis exógenas
    if constante == "S":
        try:
            exog = sm.add_constant(exog)
        except Exception: ## se não houver exógenas no modelo
            #criando um vetor de uns com o mesmo número de observações da variável endógena
            exog = np.resize([1],endog.shape[0])
            # renomeando para const
            exog = pd.DataFrame({'const':exog})
    else:
        exog = exog

    ## criando o modelo levando em conta a opção de covariância
    Modelo = IV2SLS(y,exog,endog,instrumentos)
    if cov == "robust":
        Resultado = Modelo.fit(cov_type = 'robust')
    elif cov == 'kernel': ## correlação robusta à heteroscedasticidade e autocorrelação serial
        Resultado = Modelo.fit(cov_type = 'kernel')
    elif cov == 'clustered' or cov == 'cluster':
        Resultado = Modelo.fit(cov_type = 'clustered', cluster_entity = True)
    else:
        Resultado = Modelo.fit(cov_type='unadjusted')

    print(Resultado)

## Exercício C6

In [ ]:
df = wd.data('cement')
df

,year,month,prccem,ipcem,prcpet,rresc,rnonc,ip,rdefs,milemp,...,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,1964,1,NaN,0.47425,13.400000,115401.0,142180.0,44.599998,NaN,2687.0,...,0,0,0,0,0,0,0,0,0,0
1,1964,2,NaN,0.53123,13.400000,115118.0,144190.0,45.900002,1.6634,2696.0,...,0,0,0,0,0,0,0,0,0,0
2,1964,3,NaN,0.64255,13.400000,123663.0,145577.0,46.200001,1.6626,2693.0,...,1,0,0,0,0,0,0,0,0,0
3,1964,4,NaN,0.82585,13.400000,116178.0,150793.0,46.900002,1.6626,2694.0,...,0,1,0,0,0,0,0,0,0,0
4,1964,5,NaN,1.02720,13.400000,111034.0,149259.0,47.099998,1.6610,2690.0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,1989,8,1009.0,1.28470,53.500000,164461.0,181205.0,110.500000,1.7145,NaN,...,0,0,0,0,0,1,0,0,0,0
308,1989,9,1016.0,1.19510,56.200001,162946.0,182793.0,110.900002,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
309,1989,10,1017.0,1.27390,57.599998,163195.0,178832.0,109.199997,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
310,1989,11,1016.0,NaN,57.599998,163079.0,182570.0,107.699997,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
## estimando a oferta inversa do crescimennto do preço de cimento em função do crescimento da oferta
# espera-se a1 positivo (uma maior oferta é resultado de um aumento de preço); espera-se que b1 seja positivo, já que o petróleo é um insumo para o cimento

exog = df[['gcem','gprcpet','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']]
endog = None
instrumentos = None
y = df['gprc']

Regressao_IV_MQ2E(exog=exog, endog=endog, instrumentos=instrumentos, y=y)

                            OLS Estimation Summary                            
Dep. Variable:                   gprc   R-squared:                      0.3857
Estimator:                        OLS   Adj. R-squared:                 0.3576
No. Observations:                 298   F-statistic:                    187.13
Date:                Sun, May 26 2024   P-value (F-stat)                0.0000
Time:                        22:52:01   Distribution:                 chi2(13)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0144     0.0031     4.6257     0.0000      0.0083      0.0205
gcem          -0.0443     0.0089    -4.9847     0.00

/usr/local/lib/python3.10/dist-packages/linearmodels/iv/model.py:550: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


In [ ]:
## para que gdefs seja uma boa vi de gcem, ela deve ser não correlacionada com o termo de erro e correlacionada com gcem;
# testando a correlação parcial, ve-se que a condição de relevancia não é satisfeita
exog = df[['gdefs','gprcpet','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']]
endog = None
instrumentos = None
y = df['gcem']

Regressao_IV_MQ2E(exog=exog, endog=endog, instrumentos=instrumentos, y=y)

                            OLS Estimation Summary                            
Dep. Variable:                   gcem   R-squared:                      0.8580
Estimator:                        OLS   Adj. R-squared:                 0.8517
No. Observations:                 306   F-statistic:                    1848.6
Date:                Sun, May 26 2024   P-value (F-stat)                0.0000
Time:                        22:52:01   Distribution:                 chi2(13)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -0.2482     0.0144    -17.272     0.0000     -0.2763     -0.2200
gdefs         -1.0541     3.4968    -0.3014     0.76

/usr/local/lib/python3.10/dist-packages/linearmodels/iv/model.py:550: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


In [ ]:
## testando se gres (crescimento no produto de construção residental) e gnon (crescimento no produto de construção não residencial residental) são boas VIs para gcem (crescimento da oferta de cimento)
exog = df[['gres','gnon','gprcpet','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']]
endog = None
instrumentos = None
y = df['gcem']

Regressao_IV_MQ2E(exog=exog, endog=endog, instrumentos=instrumentos, y=y)

                            OLS Estimation Summary                            
Dep. Variable:                   gcem   R-squared:                      0.8722
Estimator:                        OLS   Adj. R-squared:                 0.8661
No. Observations:                 309   F-statistic:                    2109.0
Date:                Sun, May 26 2024   P-value (F-stat)                0.0000
Time:                        22:52:01   Distribution:                 chi2(14)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -0.2437     0.0136    -17.924     0.0000     -0.2704     -0.2171
gres           0.1361     0.1350     1.0084     0.31

/usr/local/lib/python3.10/dist-packages/linearmodels/iv/model.py:550: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


In [ ]:
## o teste F é; as candidatas são boas VIs
Teste_F_Rapido_Robusto('gres = gnon = 0')

O valor de LM é 34.761842 e seu p-valor é 0.0.
Portanto, rejeita-se Ho à significância de 5.0%, ou seja, as variáveis são conjuntamente significantes.


In [ ]:
## usando gres e gnon como VI, o coeficiente de gcen é bem menos negativo e agora é estatisticamente insignificante; a curva de oferta inversa é horizontal (qualquer quantidade pode ser produzida a determinado preço)
exog = df[['gprcpet','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']]
endog = df[['gcem']]
instrumentos = df[['gres','gnon']]
y = df['gprc']

Regressao_IV_MQ2E(exog=exog, endog=endog, instrumentos=instrumentos, y=y)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   gprc   R-squared:                      0.3560
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3265
No. Observations:                 298   F-statistic:                    154.94
Date:                Sun, May 26 2024   P-value (F-stat)                0.0000
Time:                        22:52:01   Distribution:                 chi2(13)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0228     0.0071     3.2147     0.0013      0.0089      0.0367
gprcpet        0.0605     0.0154     3.9404     0.00

/usr/local/lib/python3.10/dist-packages/linearmodels/iv/model.py:550: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(


## Exercício C11

In [ ]:
df = wd.data('expendshares')

In [ ]:
## vendo a distribuição do % de gastos em comida; o mínimo é 5%, já que nenhuma pessoa não gasta nada com comida (é um bem essencial/inelástico)
df['sfood'].describe()

count    1519.000000
mean        0.356459
std         0.105125
min         0.057100
25%         0.281700
50%         0.354000
75%         0.425750
max         0.789000
Name: sfood, dtype: float64

In [ ]:
## estimando a equação; quanto maior o total de gastos, menor a proporção gasta com comida (aumento de 10% nos gastos totais diminui a proporção gast com comida em ≈1,46%)
exog = df[['ltotexpend','age','kids']]
endog = None
instrumentos = None
y = df['sfood']

Regressao_IV_MQ2E(exog=exog, endog=endog, instrumentos=instrumentos, y=y, cov='robust')

                            OLS Estimation Summary                            
Dep. Variable:                  sfood   R-squared:                      0.2862
Estimator:                        OLS   Adj. R-squared:                 0.2848
No. Observations:                1519   F-statistic:                    589.55
Date:                Sun, May 26 2024   P-value (F-stat)                0.0000
Time:                        22:52:01   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.8959     0.0291     30.750     0.0000      0.8388      0.9530
ltotexpend    -0.1459     0.0062    -23.400     0.00

In [ ]:
## usando lincome como vi de ltotexpend; lincome é muito significante, satisfazendo a condição de relevancia
exog = df[['lincome','age','kids']]
endog = None
instrumentos = None
y = df['ltotexpend']

Regressao_IV_MQ2E(exog=exog, endog=endog, instrumentos=instrumentos, y=y, cov='robust')

                            OLS Estimation Summary                            
Dep. Variable:             ltotexpend   R-squared:                      0.2563
Estimator:                        OLS   Adj. R-squared:                 0.2548
No. Observations:                1519   F-statistic:                    407.44
Date:                Sun, May 26 2024   P-value (F-stat)                0.0000
Time:                        22:52:01   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          1.9223     0.1312     14.653     0.0000      1.6652      2.1795
lincome        0.4778     0.0284     16.798     0.00

In [ ]:
## estimando a equação estrutural com lincome como vi de ltotexpend; a estimativa de ltotexpend é quase a mesma (1% mais negativa), apesar do diametro do IC ser bastante maior e conter a estimativa de MQO. De fato, parece que ltotexpend é exógena
exog = df[['age','kids']]
endog = df[['ltotexpend']]
instrumentos = df[['lincome']]
y = df['sfood']

Regressao_IV_MQ2E(exog=exog, endog=endog, instrumentos=instrumentos, y=y, cov='robust')

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  sfood   R-squared:                      0.2837
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2822
No. Observations:                1519   F-statistic:                    188.48
Date:                Sun, May 26 2024   P-value (F-stat)                0.0000
Time:                        22:52:01   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.9523     0.0540     17.629     0.0000      0.8464      1.0582
age            0.0019     0.0003     6.1526     0.00

In [ ]:
## testando se ltotexpend é exog; não podemos rejeitar a hipótese de que ltotexpend é exógena, ou seja, prefere-se estimar por MQO
## não há nenhuma restrição sobreidentificadora para ser testada
print(Resultado.wooldridge_regression)

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 1.3057
P-value: 0.2532
Distributed: chi2(1)


In [ ]:
## ao invés de sfood, usando salcohol
# MQO
exog = df[['ltotexpend','age','kids']]
endog = None
instrumentos = None
y = df['salcohol']

Regressao_IV_MQ2E(exog=exog, endog=endog, instrumentos=instrumentos, y=y, cov='robust')

                            OLS Estimation Summary                            
Dep. Variable:               salcohol   R-squared:                      0.0549
Estimator:                        OLS   Adj. R-squared:                 0.0530
No. Observations:                1519   F-statistic:                    80.623
Date:                Sun, May 26 2024   P-value (F-stat)                0.0000
Time:                        22:52:02   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0091     0.0189     0.4800     0.6312     -0.0279      0.0460
ltotexpend     0.0276     0.0041     6.6710     0.00

In [ ]:
# MQ2E; as estimativas de ltotexpend agora são positivas, indicando que o álcool é um bem não-essencial e de luxo (renda-elástico)
exog = df[['age','kids']]
endog = df[['ltotexpend']]
instrumentos = df[['lincome']]
y = df['salcohol']

Regressao_IV_MQ2E(exog=exog, endog=endog, instrumentos=instrumentos, y=y, cov='robust')

                          IV-2SLS Estimation Summary                          
Dep. Variable:               salcohol   R-squared:                      0.0547
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0529
No. Observations:                1519   F-statistic:                    53.092
Date:                Sun, May 26 2024   P-value (F-stat)                0.0000
Time:                        22:52:02   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const      -1.983e-05     0.0403    -0.0005     0.9996     -0.0789      0.0789
age           -0.0015     0.0002    -6.4359     0.00